In [13]:
from crewai import Agent, Task, LLM, Crew
from crewai.tools import tool
import os
import agentops

from langchain_groq import ChatGroq
from langchain_core.documents import Document

from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.llm import LLMChain

from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from langchain_google_community import GmailToolkit
import base64
from email.message import EmailMessage
from typing import List, Optional, Type
from langchain_core.callbacks import CallbackManagerForToolRun
from pydantic import BaseModel, Field
from langchain_google_community.gmail.base import GmailBaseTool

from dotenv import load_dotenv
from fpdf import FPDF
from PyPDF2 import PdfReader

In [14]:
load_dotenv()  

groq_key = os.getenv("GROQ_API_KEY")
agentops_key = os.getenv("AGENTOPS_API_KEY")

In [15]:
# import agentops
# agentops.init(api_key=agentops_key, auto_start_session=True)


In [16]:
outputDir = "./output"
os.makedirs(outputDir, exist_ok=True)

In [17]:
basic_llm = LLM(model="groq/llama-3.1-8b-instant", temperature=0.7, api_key=groq_key)

### Summarizer Agent

In [18]:
def read_pdf(file_path):
    reader = PdfReader(file_path)
    return "\n".join(page.extract_text() for page in reader.pages if page.extract_text())

pdf_text = read_pdf("input/sample.pdf")


In [19]:
summarizer_agent = Agent(
    role = "Summarizer Agent",
    goal = "To provide a concise summary of any given document or text, ensuring clarity, accuracy, and output in a user-friendly PDF format.",
    
    backstory = "\n".join([
            "The agent is designed to assist users in quickly understanding the main points of lengthy documents or articles.", 
            "It should distill complex information into easily digestible summaries while maintaining the original context and meaning.", 
            "The agent can also summarize weekly thoughts, reflections, and tasks to support productivity.",
            "The agent should generate a PDF file containing the summary and save it to the output directory.",
            "The agent should support multiple languages and be able to summarize content in the user's preferred language.",
            "The agent should handle various document formats, including PDFs, articles, and plain text.",
    ]),
    
    llm = basic_llm,
    verbose=True
    )


summarizer_task = Task(
    description =  f"""
        Summarize the following PDF text:{pdf_text}

        Read through the document carefully and identify the key points, main ideas, and essential information.
        Create a concise summary that captures the essence of the original text while omitting unnecessary details.
        Ensure that the summary is clear, coherent, and easy to understand.
        Generate a PDF file containing the summary and save it to the output directory.
        """,
        
    expected_output= "\n".join([
        "A well-structured summary containing a concise and accurate summary of the provided document in PDF format.",
        "The summary should be saved as a PDF file in the output directory.",
        "The summary should be clear and concise summary, formatted in short paragraphs or bullet points for easy reading."
        ]),
    agent=summarizer_agent,
    )

output_file_path = os.path.join(outputDir, "summary.pdf")

### Timetable Agent

In [20]:
timetable_agent = Agent(
    role = "Timetable Agent",
    goal = "To create a detailed weekly timetable based on user-provided tasks and priorities, ensuring optimal time management and productivity.",
    backstory = """"
            The agent is designed to assist users in organizing their weekly schedule by allocating time slots for various tasks and activities., 
            It should consider task priorities, deadlines, and estimated durations to create an efficient timetable.,
            The agent should generate a PDF file containing the timetable and save it to the output directory.,
            The agent should support multiple languages and be able to create timetables in the user's preferred language.,
            The agent should handle various input formats, including plain text, lists, and structured data (e.g., JSON).,
    """,
    llm = basic_llm,
    verbose=True
    )

timetable_task = Task(
    description= """
        Create a timetable for a whole week based on the following tasks and priorities: {schedule_text}
        Analyze each task and its priority level.
        Distribute the tasks across the week while ensuring an efficient workload balance.
        Include short breaks, rest periods, and buffer time between major tasks.
        Ensure the timetable is clear, easy to read, and organized by day and hour.
        
        Output MUST be in the following structured format (no extra text):
        Day | Time Slot | Task | Duration | Priority
        Monday | 09:00 - 11:00 | Complete project report | 2 hours | High
        Monday | 11:00 - 12:00 | Team meeting | 1 hour | Medium
        ...
    """,
    
    expected_output= """
        A well-structured weekly timetable in PDF format, saved in the output directory.
        The timetable should clearly display each day of the week with assigned time slots for all tasks.
        It should include task names, durations, priorities, and any free or break periods.
        The final PDF should be saved in the output directory as 'timetable.pdf'.  
        
        A clean, consistent table text with five columns separated by '|':
        Day | Time Slot | Task | Duration | Priority.
        No explanations or comments.
          
        """,
    agent=timetable_agent,
    )

output_file_path = os.path.join(outputDir, "timetable.pdf")

### Running the Agents

In [21]:
multi_agents = Crew(
    agents=[summarizer_agent,
            timetable_agent
            ],
    tasks=[summarizer_task,
           timetable_task
           ],
)

In [22]:
result = multi_agents.kickoff({
    "task": "Summarize this PDF",
    "input": pdf_text,
    
    
    "schedule_text": """
    
    - Task 1: Complete project report (High Priority, 2 hours)
    - Task 2: Team meeting (Medium Priority, 1 hour)
    - Task 3: Respond to emails (Low Priority, 30 minutes)""",
    
    "output_dir": "output/",
})

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Summarizer Agent                                                                                        │
│                                                                                                                 │
│  Task:                                                                                                          │
│          Summarize the following PDF text:                                                                      │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  DEBI Robot ics Challenge  2023                                                                                 │
│  Rules and Guidelines                                                                                           │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  Sponsored by:       In Cooperation with:                                                                       │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│  2/4                                                                                                            │
│                                                                                                                 │
│  Challenge  Description                                                                                         │
│  In continuation of the efforts of Digital Egypt Builders Imitative (DEBI)1, launched by the Egyptian Ministry  │
│  of                                                                                                             │
│  Communications and Information Technology , the DEBI’s Robotics & IoT Track  launches the annual               │
│  DEBI Robot ics Challenge . Inspired by the GdR TurtleB

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Summarizer Agent                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  DEBI Robotics Challenge 2023                                                                                   │
│                                                                                                                 │
│  **Challenge Description**                                                                                      │
│  The DEBI Robotics Challenge 2023 is a physical competition to advance robotic software and autonomous          │
│  capabilities. Inspired by the GdR TurtleBot Challenge 2018, the challenge is a one-vs-one match in a           │
│  predefined playground, where robots must reduce the number of balls in their area by moving them to the        │
│  opponent's side.                                                                                               │
│                                                                                                                 │
│  **Competition Rules**                                                                                          │
│                                                                                                                 │
│  * The competition will take place on May 13, 2023, at DEBI's campus in Cairo.                                  │
│  * The competition is based on knockout matches, with the winner determined by the team with the fewest balls   │
│  on their side.                                                                                                 │
│  * Robots must be fully autonomous and cannot step onto or pass the red line or playground walls.               │
│  * If a robot crosses the boundaries of its side of the playground, it must start over from the initial         │
│  position.                                                                                                      │
│  * The match duration is 10 minutes, with a 3-minute extension if no team wins.                                 │
│                                                                                                                 │
│  **Rewards**                                                                                                    │
│  The winning teams will receive a financial award:                                                              │
│  * First place: 80,000 EGP                                                                                      │
│  * Second place: 60,000 EGP                                                                                     │
│  * Third place: 40,000 EGP                                                                                      │
│                                                                                                                 │
│  **Eligibility Criteria and Teams Formation**                                                                   │
│  * All participants must hold Egyptian nationality.                                                             │
│  * All participants must be registered undergraduate students at any Egyptian university.                       │
│  * Graduates from no more than 5 years ago are allowed to participate.                                          │
│  * Each team consists of 3-5 members, with a designated team leader.                                            │
│                                                        

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Timetable Agent                                                                                         │
│                                                                                                                 │
│  Task:                                                                                                          │
│          Create a timetable for a whole week based on the following tasks and priorities:                       │
│                                                                                                                 │
│      - Task 1: Complete project report (High Priority, 2 hours)                                                 │
│      - Task 2: Team meeting (Medium Priority, 1 hour)                                                           │
│      - Task 3: Respond to emails (Low Priority, 30 minutes)                                                     │
│          Analyze each task and its priority level.                                                              │
│          Distribute the tasks across the week while ensuring an efficient workload balance.                     │
│          Include short breaks, rest periods, and buffer time between major tasks.                               │
│          Ensure the timetable is clear, easy to read, and organized by day and hour.                            │
│                                                                                                                 │
│          Output MUST be in the following structured format (no extra text):                                     │
│          Day | Time Slot | Task | Duration | Priority                                                           │
│          Monday | 09:00 - 11:00 | Complete project report | 2 hours | High                                      │
│          Monday | 11:00 - 12:00 | Team meeting | 1 hour | Medium                                                │
│          ...                                                                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Timetable Agent                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Monday | 09:00 - 11:00 | Complete project report | 2 hours | High                                              │
│  Monday | 11:00 - 12:00 | Team meeting | 1 hour | Medium                                                        │
│  Monday | 12:00 - 12:30 | Short break | 0.5 hours |                                                             │
│  Tuesday | 09:00 - 10:00 | Respond to emails | 0.5 hours | Low                                                  │
│  Tuesday | 10:00 - 12:00 | Task 4: Research project | 2 hours | Medium                                          │
│  Tuesday | 12:00 - 13:00 | Lunch break | 1 hour |                                                               │
│  Wednesday | 09:00 - 11:00 | Task 5: Meet with client | 2 hours | High                                          │
│  Wednesday | 11:00 - 12:00 | Task 6: Team discussion | 1 hour | Medium                                          │
│  Wednesday | 12:00 - 13:00 | Short break | 1 hour |                                                             │
│  Thursday | 09:00 - 10:00 | Task 7: Review reports | 1 hour | Medium                                            │
│  Thursday | 10:00 - 12:00 | Task 8: Analysis | 2 hours | High                                                   │
│  Thursday | 12:00 - 13:00 | Lunch break | 1 hour |                                                              │
│  Friday | 09:00 - 10:00 | Task 9: Meeting with team | 1 hour | Medium                                           │
│  Friday | 10:00 - 12:00 | Task 10: Project planning | 2 hours | High                                            │
│  Friday | 12:00 - 13:00 | Short break | 1 hour |                                                                │
│                                                                                                                 │
│  [PDF File saved to output directory]                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [23]:

# # Save result to PDF

# summary_text = result.raw   # string output

# pdf = FPDF()
# pdf.add_page()
# pdf.set_font("Arial", size=12)

# # Split summary text into lines
# for line in summary_text.split("\n"):
#     pdf.multi_cell(0, 10, line)

# # Save to output folder
# output_file_path = os.path.join("output", "summary.pdf")
# os.makedirs("output", exist_ok=True)
# pdf.output(output_file_path)


# print(f"Summary saved at {os.path.abspath(output_file_path)}")

In [24]:
# def save_pdf(text, filename):
#     pdf = FPDF()
#     pdf.add_page()
#     pdf.set_font("Arial", size=12)
#     for line in text.split("\n"):
#         pdf.multi_cell(0, 10, line)
#     os.makedirs("output", exist_ok=True)
#     pdf.output(os.path.join("output", filename))


In [25]:
def save_output(text, filename, table=False):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=11)

    if table:
        lines = [line.strip() for line in text.split("\n") if line.strip() and "|" in line]

        headers = ["Day", "Time Slot", "Task", "Duration", "Priority"]
        col_widths = [30, 40, 70, 25, 25]

        pdf.set_fill_color(200, 200, 200)
        pdf.set_font("Arial", "B", 11)
        for i, header in enumerate(headers):
            pdf.cell(col_widths[i], 10, header, border=1, align="C", fill=True)
        pdf.ln()

        pdf.set_font("Arial", size=10)

        for line in lines:
            parts = [p.strip() for p in line.split("|")]
            if len(parts) == 5:
                for i, val in enumerate(parts):
                    val_clean = val.encode('latin-1', 'replace').decode('latin-1')
                    pdf.cell(col_widths[i], 8, val_clean, border=1, align="C")
                pdf.ln()
    else:
        # Strip markdown formatting
        import re
        clean_text = text
        
        # Remove markdown bold/italic
        clean_text = re.sub(r'\*\*\*(.+?)\*\*\*', r'\1', clean_text)  # ***text***
        clean_text = re.sub(r'\*\*(.+?)\*\*', r'\1', clean_text)      # **text**
        clean_text = re.sub(r'\*(.+?)\*', r'\1', clean_text)          # *text*
        clean_text = re.sub(r'__(.+?)__', r'\1', clean_text)          # __text__
        clean_text = re.sub(r'_(.+?)_', r'\1', clean_text)            # _text_
        
        # Replace special characters
        clean_text = (clean_text
                     .replace('•', '- ')
                     .replace('–', '-')
                     .replace('—', '-')
                     .replace(''', "'")
                     .replace(''', "'")
                     .replace('"', '"')
                     .replace('"', '"')
                     .replace('…', '...')
                     .replace('★', '*')
                     )
        
        # Process line by line
        for line in clean_text.split("\n"):
            # Clean and strip the line
            line_clean = line.encode('latin-1', 'ignore').decode('latin-1').strip()
            
            if not line_clean:
                pdf.ln(5)  # Add small space for empty lines
                continue
            
            # Check if this is a header (title) - make it bold
            if line_clean.isupper() or (len(line_clean) < 100 and line == line.strip() and not line.startswith('-')):
                pdf.set_font("Arial", "B", 12)
                pdf.cell(0, 10, line_clean, ln=True)
                pdf.set_font("Arial", size=11)
            else:
                # Regular text - use cell instead of multi_cell for better control
                # Break long lines manually
                max_width = 180  # Effective page width
                char_width = 2.5  # Approximate character width in mm
                max_chars = int(max_width / char_width)
                
                if len(line_clean) <= max_chars:
                    pdf.cell(0, 6, line_clean, ln=True)
                else:
                    # Break into chunks
                    words = line_clean.split()
                    current_line = ""
                    
                    for word in words:
                        test_line = current_line + (" " if current_line else "") + word
                        if len(test_line) <= max_chars:
                            current_line = test_line
                        else:
                            if current_line:
                                pdf.cell(0, 6, current_line, ln=True)
                            current_line = word
                    
                    if current_line:
                        pdf.cell(0, 6, current_line, ln=True)

    os.makedirs("output", exist_ok=True)
    output_path = os.path.join("output", filename)
    pdf.output(output_path)
    print(f"PDF saved: {output_path}")

In [26]:
save_output(result.tasks_output[0].raw, "summary.pdf")  
save_output(result.tasks_output[1].raw, "timetable.pdf", table=True)

PDF saved: output\summary.pdf
PDF saved: output\timetable.pdf


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2972\3569336528.py:4: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", size=11)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2972\3569336528.py:63: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", "B", 12)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2972\3569336528.py:64: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, line_clean, ln=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2972\3569336528.py:65: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", size=11)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2972\3569336528.py:86: DeprecationWarn

## LangChain

In [27]:
llm = ChatGroq(
    model_name="llama-3.1-8b-instant",
    temperature=0.7
)

### Summarizer Chain - LangChain

In [28]:
# create_stuff_documents_chain is specifically designed for document summarization --> returns a string
document = Document(page_content=pdf_text, metadata={"source": "sample.pdf"})
template_string = "Summarize this content: {context}"
prompt = ChatPromptTemplate.from_template(template_string)
chain = create_stuff_documents_chain(llm, prompt)

In [29]:
result = chain.invoke({"context": [document]})
result

"The DEBI Robotics Challenge 2023 is a physical competition sponsored by the Egyptian Ministry of Communications and Information Technology, launched by the Digital Egypt Builders Initiative (DEBI). The competition is a one-vs-one match between robots in a predefined playground, where the objective is to move balls from one side to the other side.\n\n**Key Rules and Guidelines:**\n\n1. The competition will take place on May 13, 2023, at DEBI's campus in Cairo.\n2. Each team must consist of 3-5 members, all of whom must be registered undergraduate students at an Egyptian university.\n3. Teams must register online by April 2, 2023.\n4. Robots must be fully autonomous and cannot cross the red line or playground walls.\n5. The match duration is 10 minutes, with a possible 3-minute extension if no team wins.\n6. The winning team will receive a financial award of 80,000 EGP (first place), 60,000 EGP (second place), or 40,000 EGP (third place).\n\n**Robot Specifications:**\n\n1. DEBI will pro

In [30]:
# The result from create_stuff_documents_chain is already a string
save_output(result, "langchain_summary.pdf")

PDF saved: output\langchain_summary.pdf


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2972\3569336528.py:4: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", size=11)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2972\3569336528.py:86: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 6, current_line, ln=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2972\3569336528.py:90: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 6, current_line, ln=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2972\3569336528.py:63: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", "B", 12)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2972\3569336528.py:64: Depreca

### Timetable Chain - LangChain

In [31]:
schedule_text = """
    
    - Task 1: Complete project report (High Priority, 2 hours)
    - Task 2: Team meeting (Medium Priority, 1 hour)
    - Task 3: Respond to emails (Low Priority, 30 minutes)"""


In [32]:
# LLMChain is a more general-purpose chain --> returns a dictionary
prompt = ChatPromptTemplate.from_template("""Make a timetable for a whole week for the following tasks: {schedule_text})

Output MUST be in the following structured format (no extra text):
Day | Time Slot | Task | Duration | Priority
Example:
Monday | 09:00 - 11:00 | Complete project report | 2 hours | High
Monday | 11:00 - 12:00 | Team meeting | 1 hour | Medium""")

chain = LLMChain(llm=llm, prompt=prompt)
result = chain.invoke({"schedule_text": schedule_text})
result

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2972\293944964.py:10: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


{'schedule_text': '\n\n    - Task 1: Complete project report (High Priority, 2 hours)\n    - Task 2: Team meeting (Medium Priority, 1 hour)\n    - Task 3: Respond to emails (Low Priority, 30 minutes)',
 'text': 'Monday | 09:00 - 11:00 | Complete project report | 2 hours | High\nMonday | 11:00 - 12:00 | Team meeting | 1 hour | Medium\nMonday | 12:00 - 12:30 | Respond to emails | 0.5 hours | Low\n\nTuesday | 09:00 - 09:30 | Respond to emails | 0.5 hours | Low\nTuesday | 09:30 - 11:30 | Complete project report | 2 hours | High\n\nWednesday | 09:00 - 10:00 | Team meeting | 1 hour | Medium\nWednesday | 10:00 - 10:30 | Respond to emails | 0.5 hours | Low\n\nThursday | 09:00 - 11:00 | Complete project report | 2 hours | High\nThursday | 11:00 - 12:00 | Team meeting | 1 hour | Medium\nThursday | 12:00 - 12:30 | Respond to emails | 0.5 hours | Low\n\nFriday | 09:00 - 09:30 | Respond to emails | 0.5 hours | Low\nFriday | 09:30 - 11:30 | Complete project report | 2 hours | High\n\nSaturday | 10:0

In [33]:
save_output(result['text'], "langchain_timetable.pdf", table=True)

PDF saved: output\langchain_timetable.pdf


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2972\3569336528.py:4: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", size=11)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2972\3569336528.py:13: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", "B", 11)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2972\3569336528.py:18: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", size=10)


### Email Drafting Agent

In [34]:
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

def main():
    creds = None
    flow = InstalledAppFlow.from_client_secrets_file(
        "credentials.json", SCOPES
    )
    creds = flow.run_local_server(port=0)

    service = build("gmail", "v1", credentials=creds)
    results = service.users().labels().list(userId="me").execute()
    labels = results.get("labels", [])

    print("Labels:")
    for label in labels:
        print(label["name"])

if __name__ == "__main__":
    main()


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=734242446147-2juvs3u82eb8n3oocp5l55nc3ari52d9.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A62344%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=aq6yXmvccKSnB67InlnJ5XAK31so8H&access_type=offline
Labels:
CHAT
SENT
INBOX
IMPORTANT
TRASH
DRAFT
SPAM
CATEGORY_FORUMS
CATEGORY_UPDATES
CATEGORY_PERSONAL
CATEGORY_PROMOTIONS
CATEGORY_SOCIAL
YELLOW_STAR
STARRED
UNREAD


In [35]:
toolkit = GmailToolkit()

In [36]:

class CreateDraftSchema(BaseModel):
    """Input for CreateDraftTool."""

    message: str = Field(...,description="The message to include in the draft.",)
    to: List[str] = Field(...,description="The list of recipients.",)
    subject: str = Field(...,description="The subject of the message.",)
    cc: Optional[List[str]] = Field(default=None,description="The list of CC recipients.",)
    bcc: Optional[List[str]] = Field(default=None,description="The list of BCC recipients.",)



class GmailCreateDraft(GmailBaseTool):
    """Tool that creates a draft email for Gmail."""

    name: str = "create_gmail_draft"
    description: str = (
        "Use this tool to create a draft email with the provided message fields."
    )
    args_schema: Type[CreateDraftSchema] = CreateDraftSchema

    def _prepare_draft_message(
        self,
        message: str,
        to: List[str],
        subject: str,
        cc: Optional[List[str]] = None,
        bcc: Optional[List[str]] = None,
    ) -> dict:
        draft_message = EmailMessage()
        draft_message.set_content(message)

        draft_message["To"] = ", ".join(to)
        draft_message["Subject"] = subject
        if cc is not None:
            draft_message["Cc"] = ", ".join(cc)

        if bcc is not None:
            draft_message["Bcc"] = ", ".join(bcc)

        encoded_message = base64.urlsafe_b64encode(draft_message.as_bytes()).decode()
        return {"message": {"raw": encoded_message}}

    def _run(
        self,
        message: str,
        to: List[str],
        subject: str,
        cc: Optional[List[str]] = None,
        bcc: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        try:
            create_message = self._prepare_draft_message(message, to, subject, cc, bcc)
            draft = (
                self.api_resource.users()
                .drafts()
                .create(userId="me", body=create_message)
                .execute()
            )
            output = f"Draft created. Draft Id: {draft['id']}"
            return output
        except Exception as e:
            raise Exception(f"An error occurred: {e}")


In [37]:
creds = Credentials.from_authorized_user_file(
    "token.json", 
    ["https://www.googleapis.com/auth/gmail.compose"]
)
service = build("gmail", "v1", credentials=creds)

tool = GmailCreateDraft(api_resource=service)

result = tool.run({
    "message": "This is a test draft from LangChain.",
    "to": ["example@gmail.com"],
    "subject": "Hello again from Agent"
})

print(result)


Draft created. Draft Id: r-8023465393060581391
